# 02. Advanced Ollama Setup

## 1. Introduction

In this notebook, we'll set up multiple Ollama instances in Docker containers. Ollama is an open-source tool that allows us to run large language models locally. This setup will enable us to use different models for various tasks in our RAG (Retrieval-Augmented Generation) system, such as embedding generation and text generation.

We'll cover the following steps:
1. Updating our project directory structure
2. Creating an OllamaManager class to handle Ollama operations
3. Updating our environment variables
4. Updating our Docker Compose configuration
5. Testing the OllamaManager class

## 2. Update Project Directory Structure

First, let's update our project directory structure to accommodate the Ollama models and ensure they're shared between containers.

Our updated project structure will now looks like this:

```
ProjectName/
├── config/
│   ├── docker-compose.yml
│   └── .env
├── notebooks/
│   ├── 00_Environment_Setup.ipynb
│   ├── 01_Database_Setup.ipynb
│   └── 02_Ollama_setup.ipynb
├── src/
│   └── utils/
│       ├── config_utils.py
│       └── ollama_manager.py
├── db_data/
│   ├── postgres/
│   ├── neo4j/
│   └── ollama_models/
│       └── llm/
└── tests/
```

In [ ]:
import os

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Create directories for Ollama models
ollama_models_path = os.path.join(project_root, 'db_data', OLLAMA_models')
os.makedirs(ollama_models_path, exist_ok=True)

print(f"Created Ollama models directory at: {ollama_models_path}")

# Instead of creating a specific CodeLlama directory, we'll create a function to make model-specific directories as we need them.
def create_model_directory(model_name):
    model_path = os.path.join(ollama_models_path, model_name)
    os.makedirs(model_path, exist_ok=True)
    print(f"Created {model_name} directory at: {model_path}")


This setup provides a flexible structure for managing multiple Ollama models. The `create_model_directory` function will be used later in this notebook for the OllamaManager class.

**Example usage:**
```python
create_model_directory('codellama')
```

Explanation of changes:
- We've made the directory creation more generic to support multiple models.
- We've created a function `create_model_directory` to easily create directories for new models.

The `create_model_directory` function takes a model name as an argument and creates a dedicated directory for that model within the `ollama_models_path`. This allows us to organize model-specific data and configurations separately, which will be crucial when working with multiple models in our RAG system.

## 3. Update Environment Variables

Now, let's update our .env file with the necessary Ollama configurations. These environment variables will serve as the foundation for our OllamaManager class and Docker setup:

In [ ]:
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Ollama environment variables to append
ollama_env_vars = """
# Ollama Configuration
OLLAMA_MODELS_PATH=./db_data/ollama_models

# Ollama Configuration
OLLAMA_MODELS_PATH=./db_data/ollama_models

# CodeLlama Configuration
OLLAMA_CODESTRALL_CONTAINER_NAME=ragtools_ollama_codestral
OLLAMA_CODESTRALL_PORT=11435
OLLAMA_CODESTRALL_MODEL=sammcj/codestral-tweaked-22b
OLLAMA_CODESTRALL_PATH=../db_data/ollama_models/codestral
OLLAMA_CODESTRALL_GPU=0
OLLAMA_CODESTRALL_HOST=0.0.0.0

# Example of another model configuration (commented out)
# OLLAMA_LLAMA2_CONTAINER_NAME=ragtools_ollama_llama2
# OLLAMA_LLAMA2_PORT=11436
# OLLAMA_LLAMA2_MODEL=llama2
# OLLAMA_LLAMA2_PATH=./db_data/ollama_models/llama2
# OLLAMA_LLAMA2_GPU=1  # Assign to second GPU
"""

# Path to the .env file
env_file_path = os.path.join('..', '..', 'config', '.env')

try:
    # Check if file exists and is writable
    if os.path.exists(env_file_path):
        if os.access(env_file_path, os.W_OK):
            with open(env_file_path, 'a') as f:
                f.write(ollama_env_vars)
            logger.info(f"Successfully appended Ollama configurations to .env file at {env_file_path}")
        else:
            logger.error(f"The .env file at {env_file_path} is not writable.")
    else:
        logger.error(f".env file not found at {env_file_path}. Please create it first.")

except IOError as e:
    logger.error(f"An error occurred while writing to the .env file: {e}")

print("Ollama environment variable setup completed.")


This code appends the Ollama-specific configurations to our existing .env file. These variables will be used in our Docker Compose file to set up the Ollama services.

## 4. Review Docker Compose Configuration

Our docker-compose.yml file now includes the following Ollama service configuration:

```yaml
ollama_codestral:
  image: ollama/ollama
  container_name: ${OLLAMA_CODESTRALL_CONTAINER_NAME:-ragtools_ollama_codestral}
  environment:
    - OLLAMA_HOST=0.0.0.0:${OLLAMA_CODESTRALL_PORT:-11435}
  env_file:
    - .env
  ports:
    - ${OLLAMA_CODESTRALL_PORT:-11435}:${OLLAMA_CODESTRALL_PORT:-11435}
  volumes:
    - ../db_data/ollama_models:/root/.ollama
    - ../db_data/ollama_models/codestral:/root/.ollama/codestral
  entrypoint: ["ollama"]
  command: ["serve"]
  deploy:
    resources:
      reservations:
        devices:
          - driver: nvidia
            count: 1
            capabilities:
              - gpu
  networks:
    - ragtools_network
```

This configuration ensures that:
1. The Ollama service uses the correct port as specified in the .env file.
2. The OLLAMA_HOST environment variable is set correctly.
3. The service uses GPU capabilities if available.
4. The correct volumes are mounted for persistent storage of models.

**NOTE**: run the next code block to update the docker compose file.

In [ ]:
ollama_service_config = """
    ollama_codestral:
        image: ollama/ollama
        container_name: ${OLLAMA_CODESTRALL_CONTAINER_NAME:-ragtools_ollama_codestral}
        environment:
            - OLLAMA_HOST=0.0.0.0:${OLLAMA_CODESTRALL_PORT:-11435}
        env_file:
            - .env
        ports:
            - ${OLLAMA_CODESTRALL_PORT:-11435}:${OLLAMA_CODESTRALL_PORT:-11435}
        volumes:
            - ../db_data/ollama_models:/root/.ollama
            - ../db_data/ollama_models/codestral:/root/.ollama/codestral
        entrypoint: ["ollama"]
        command: ["serve"]
        deploy:
            resources:
            reservations:
                devices:
                - driver: nvidia
                    count: 1
                    capabilities:
                    - gpu
        networks:
            - ragtools_network
"""

print("Ollama service configuration in docker-compose.yml:")
print(ollama_service_config)


## 5. Update Config Class

Before we proceed with the verification step, we need to update our Config class to include the new Ollama-related attributes. This is a crucial step when extending our framework with new components.

This step demonstrates how to extend the Config class when new components are added to the framework. It's important to update this class whenever new environment variables or configuration options are introduced.

Let's update the `config_utils.py` file:

In [ ]:
import os

config_utils_path = os.path.join('..', 'src', 'utils', 'config_utils.py')

# Read the existing content
with open(config_utils_path, 'r') as f:
    existing_content = f.read()

# Define the new Ollama configurations
ollama_configs = '''
        # Ollama configurations
        self.OLLAMA_MODELS_PATH = os.getenv('OLLAMA_MODELS_PATH')
        
        # CodeStral Configuration
        self.OLLAMA_CODESTRALL_CONTAINER_NAME = os.getenv('OLLAMA_CODESTRALL_CONTAINER_NAME')
        self.OLLAMA_CODESTRALL_PORT = int(os.getenv('OLLAMA_CODESTRALL_PORT', 11435))
        self.OLLAMA_CODESTRALL_MODEL = os.getenv('OLLAMA_CODESTRALL_MODEL')
        self.OLLAMA_CODESTRALL_PATH = os.getenv('OLLAMA_CODESTRALL_PATH')
        self.OLLAMA_CODESTRALL_GPU = int(os.getenv('OLLAMA_CODESTRALL_GPU', 0))
        self.OLLAMA_CODESTRALL_HOST = os.getenv('OLLAMA_CODESTRALL_HOST')
'''

# Find the position to insert the new configurations
lines = existing_content.split('\n')
insert_line = -1
for i, line in enumerate(lines):
    if line.strip().startswith('def get_postgres_connection_params(self):'):
        insert_line = i
        break

if insert_line == -1:
    # If method not found, insert at the end of __init__
    for i, line in enumerate(reversed(lines)):
        if line.strip() == "self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')":
            insert_line = len(lines) - i
            break

# Insert the new configurations
if insert_line != -1:
    updated_lines = lines[:insert_line] + ollama_configs.split('\n') + lines[insert_line:]
    updated_content = '\n'.join(updated_lines)
else:
    print("Could not find appropriate insertion point. Please update manually.")
    updated_content = existing_content

# Write the updated content back to the file
with open(config_utils_path, 'w') as f:
    f.write(updated_content)

print("Updated config_utils.py with Ollama configurations.")

# Optionally, print the updated Config class for verification
print("\nUpdated Config class:")
print(updated_content)


## 6. ## Testing Ollama Setup

Now that we have configured our Ollama service in the Docker Compose file and added an entrypoint script, let's test it to ensure everything is working correctly. For these steps, you'll need to open a terminal or command prompt and navigate to the directory containing your docker-compose.yml file.

### 1. Ensure you're in the correct directory

First, make sure you're in the directory containing your docker-compose.yml file:

```bash
cd path/to/your/project/config
```

### 2. Start the Docker containers:

Run the following command to start your Docker containers:

```bash
docker compose --env-file .env up -d
```

You should see output indicating that the containers are starting, including the Ollama container.

### 3. Verify that the Ollama container is running:

```bash
docker ps | grep ollama
```

You should see output similar to:

```
3a3b21e26914   ollama/ollama   "/bin/bash /entrypoint.sh"   2 minutes ago   Up 2 minutes   11434/tcp, 0.0.0.0:11435->11435/tcp, :::11435->11435/tcp   ragtools_ollama_codestrall
```

### 4. Check if the Ollama service is responsive:

```bash
curl http://localhost:11435/api/tags
```

This should return a JSON response with available models. If it's empty ({}), we need to pull the Codestrall model.

### 5. Pull the Codestrall model:

If the model isn't already pulled, run:

```bash
docker exec ragtools_ollama_codestrall ollama pull sammcj/codestral-tweaked-22b
```

This may take some time depending on your internet connection and system resources.

### 6. Test the model with a simple prompt:

After the model is pulled, you can test it with:

```bash
curl -X POST http://localhost:11435/api/generate -d '{
  "model": "sammcj/codestral-tweaked-22b",
  "prompt": "Explain what RAG stands for in the context of AI:"
}'
```

This should return a JSON response containing the generated text explaining RAG (Retrieval-Augmented Generation).

### 7. (Optional) Log into the Ollama container:

If you need to perform any operations inside the container:

```bash
docker exec -it ragtools_ollama_codestrall /bin/bash
```

Once inside, you can run Ollama commands directly:

```bash
ollama run sammcj/codestral-tweaked-22b "Explain what RAG stands for in the context of AI:"
```

Exit the container when done:

```bash
exit
```

### Stopping the Containers

When you're done testing, you can stop the Docker containers:

```bash
docker compose down
```

By performing these tests, we ensure that our Ollama service is correctly set up and functioning as expected with the sammcj/codestral-tweaked-22b model. The entrypoint script ensures that the Ollama service starts correctly within the container. This setup lays the groundwork for creating our OllamaManager class in the next section, which will provide a more structured way to interact with Ollama in our RAG system.

**Note**: If you encounter any issues running these commands, ensure that Docker is properly installed and running on your system, and that you have the necessary permissions to execute Docker commands.


## Conclusion

In this notebook, we have successfully:

1. Set up Ollama instances in Docker containers
2. Created an OllamaManager class to handle Ollama operations
3. Implemented a method to generate responses from the LLM
4. Demonstrated the streaming nature of the LLM's output
5. Verified the functionality of our setup with test questions

## Next Steps

Our next notebook will focus on creating a CLI interface for interacting with the LLM. Before diving into the implementation, we'll need to consider:

1. LLM Configurables:
   - Context length
   - Temperature
   - Other relevant parameters (e.g., top_p, frequency_penalty, presence_penalty)

2. CLI Interface Options:
   - Evaluate the merits of adopting a pre-built CLI interface vs. creating our own
   - Consider libraries like `click`, `typer`, or `argparse` for building a custom CLI

3. Chat Interface Design:
   - How to maintain conversation history
   - Handling user input and system responses
   - Implementing commands for adjusting LLM parameters on-the-fly

4. Integration with OllamaManager:
   - How to incorporate our existing OllamaManager class into the CLI interface

By addressing these points, we'll be well-prepared to create a robust and user-friendly CLI for interacting with our LLM setup.